In [2]:
# Importar paquetes necesarios
import folium
import pandas as pd
import requests
from io import BytesIO
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians


In [3]:
# Descargar y leer el conjunto de datos usando requests
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_csv_file = BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

In [4]:
# Seleccionar columnas relevantes
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Ubicación inicial es el Centro Espacial Johnson de la NASA
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)


# Tarea 1: Para cada sitio de lanzamiento, agregar un Círculo y un Marcador

In [5]:
# Tarea 1: Para cada sitio de lanzamiento, agregar un Círculo y un Marcador
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle(
        [site['Lat'], site['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    
    marker = folium.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)


In [6]:
# Guardar el mapa para la Tarea 1
site_map.save('task1_launch_sites.html')

# Tarea 2: Marcar los lanzamientos exitosos/fallidos para cada sitio

In [7]:
# Tarea 2: Marcar los lanzamientos exitosos/fallidos para cada sitio
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Sitio de Lanzamiento: {record['Launch Site']}<br>Resultado: {'Éxito' if record['class'] == 1 else 'Fallo'}"
    )
    marker_cluster.add_child(marker)


In [8]:
# Guardar el mapa para la Tarea 2
site_map.save('task2_launch_outcomes.html')

# Tarea 3: Calcular distancias entre sitios de lanzamiento y sus proximidades

In [9]:
# Función para calcular distancia entre dos coordenadas
def calculate_distance(lat1, lon1, lat2, lon2):
    # radio aproximado de la tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Agregar Posición del Ratón para obtener coordenadas
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Coordenadas de los puntos de costa más cercanos a cada sitio de lanzamiento
coastline_points = {
    'CCAFS LC-40': [28.56342, -80.5680],
    'CCAFS SLC-40': [28.56342, -80.5680],
    'KSC LC-39A': [28.58333, -80.58333],
    'VAFB SLC-4E': [34.63533, -120.62492]
}

In [10]:
# Para cada sitio de lanzamiento, calcular distancia a la costa y agregar marcadores
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat = site['Lat']
    site_lon = site['Long']
    coastline_lat, coastline_lon = coastline_points[site_name]
    
    # Calcular distancia
    distance = calculate_distance(site_lat, site_lon, coastline_lat, coastline_lon)
    
    # Crear marcador para el punto de costa
    distance_marker = folium.Marker(
        [coastline_lat, coastline_lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
        )
    )
    
    # Dibujar una línea entre el sitio de lanzamiento y la costa
    lines = folium.PolyLine(
        locations=[[site_lat, site_lon], [coastline_lat, coastline_lon]],
        weight=1
    )
    
    site_map.add_child(distance_marker)
    site_map.add_child(lines)

# Guardar el mapa final con todas las visualizaciones
site_map.save('spacex_launch_sites_analysis.html')

print("Análisis completado. Los mapas se han guardado como:")
print("1. task1_launch_sites.html - Sitios de lanzamiento")
print("2. task2_launch_outcomes.html - Resultados de lanzamientos")
print("3. spacex_launch_sites_analysis.html - Análisis completo con distancias")

Análisis completado. Los mapas se han guardado como:
1. task1_launch_sites.html - Sitios de lanzamiento
2. task2_launch_outcomes.html - Resultados de lanzamientos
3. spacex_launch_sites_analysis.html - Análisis completo con distancias
